In [1]:
from time import sleep, strftime
from random import randint
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import smtplib
from datetime import *
from __future__ import division, print_function  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests 
from bs4 import BeautifulSoup as bs
sns.set()

%matplotlib inline
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression

from sklearn.model_selection import (cross_val_score, train_test_split, 
                                     KFold, GridSearchCV)
import statsmodels.api as sm

In [2]:
chromedriver = "/Applications/chromedriver"

### This is my function that starts selenium, grabs each edibility type, and calls the scraping funcitons defined below 

Each Type refers to edibility type. The num defined after each type is the number of pages selenium needs to iterate through for that edibility type.

In [97]:
def start_mushrooms(type1,num1,type2,num2,type3,num3):
    mushroom_posionous = ('https://www.mushroom.world/mushrooms/'+ type1)
    driver.get(mushroom_posionous)
    sleep(10)

    accept_cookies()
    accept_warning()
    
    mushroom_posionous_df = pd.DataFrame()
    mushroom_posionous_img_df = pd.DataFrame()
    mushroom_inedible_df = pd.DataFrame()
    mushroom_inedible_img_df = pd.DataFrame()
    mushroom_edible_df = pd.DataFrame()
    mushroom_edible_img_df = pd.DataFrame()
    
    for i in range (num1-1):
        mushroom_posionous_df = mushroom_posionous_df.append(scrape())
        mushroom_posionous_img_df = mushroom_posionous_img_df.append(scrape_img())
        next_page(i, type1, num1-1)

    mushroom_posionous_df = mushroom_posionous_df.append(scrape()) # This grabs the info from the last page after it is loaded and the for loop has ended
    mushroom_posionous_img_df = mushroom_posionous_img_df.append(scrape_img()) # This grabs the info from the last page after it is loaded and the for loop has ended
    
    mushroom_posionous_img_df['edibility'] = 'posionous'
    
    print('switching to inedible')
    mushroom_inedible = ('https://www.mushroom.world/mushrooms/'+ type2)
    driver.get(mushroom_inedible)
    
    for i in range (num2-1):
        mushroom_inedible_df = mushroom_inedible_df.append(scrape())
        mushroom_inedible_img_df = mushroom_inedible_img_df.append(scrape_img())
        next_page(i, type2, num2-1)
    
    mushroom_inedible_df = mushroom_inedible_df.append(scrape())
    mushroom_inedible_img_df = mushroom_inedible_img_df.append(scrape_img())
    
    mushroom_inedible_img_df['edibility'] = 'inedible'
    
    print('switching to edible')
    mushroom_edible = ('https://www.mushroom.world/mushrooms/'+ type3)
    driver.get(mushroom_edible)  
    
    for i in range (num3-1):
        mushroom_edible_df = mushroom_edible_df.append(scrape())
        mushroom_edible_img_df = mushroom_edible_img_df.append(scrape_img())
        next_page(i, type3, num3-1)
    
    mushroom_edible_df = mushroom_edible_df.append(scrape())
    mushroom_edible_img_df = mushroom_edible_img_df.append(scrape_img())
    
    mushroom_edible_img_df['edibility'] = 'edible'

    final_df_img = mushroom_posionous_img_df.append(mushroom_edible_img_df).append(mushroom_inedible_img_df)
    final_df = mushroom_posionous_df.append(mushroom_edible_df).append(mushroom_inedible_df)
    final_df.to_csv('mushrooms.csv')
    final_df_img.to_csv('mushrooms_img.csv')
    print('Final Data Frames Saved')

#### These functions below help me navigate specific items on the web page 

In [4]:
def accept_cookies():
        accept_cookies = driver.find_element_by_xpath('//*[text() = "Accept and continue"]')
        accept_cookies.click()
        print('cookies accepted')
        sleep(randint(15,25))

In [5]:
def accept_warning():
        accept_warning = driver.find_element_by_xpath('//input[@value = "Got it!"][@type = "button"]')
        accept_warning.click()
        print('warning accepted')
        sleep(randint(15,25))

In [6]:
def next_page(i,edibility, num):
    next_page = driver.find_element_by_xpath('//div[@id = "pager"]/a[text() = " Next Page »"]')
    next_page.click()
    print('scraping',edibility, 'page',i+1,'out of',num)
    sleep(randint(25,35))

### Below are the main scraping functions. I scraped data into both a text data frame and an image data frame due to entry size differences in the data types  

In [84]:
def scrape():
    html=driver.page_source
    soup=bs(html)
    
    results = soup.find(id="mushroom-list")

    
    Species_Names = results.find_all("div",class_="caption")
    Short_Text = results.find_all("div", class_="textus")
    Descriptions = results.find_all("div", class_="longtextus")
    

    Species_Name = []
    Family = []
    Location = []
    Dimensions = []
    Edibility = []
    Description = []
    
    

    holder = []
    for sn in Species_Names:
        Species_Name.append((sn).text)
        
    for d in Descriptions:
        Description.append((d).text)
    
    for s in range(len(Short_Text)):
        holder.append(Short_Text[s].text)
        Family = holder[::5]
        Location = holder[1::5]
        Dimensions = holder[2::5]
        Edibility = holder[3::5]

    df = pd.DataFrame((zip(Species_Name, Family, Location, Dimensions, Edibility, Description)),
                     columns = ['Species_Name','Family','Location','Dimensions','Edibility','Description'])

    return (df)

In [85]:
def scrape_img():
    html=driver.page_source
    soup=bs(html,'html.parser')

    Images_List = []
    
    imageshtml = soup.find_all('img')
    
    for item in imageshtml:
        Images_List.append(item['src'])

    df_img=pd.DataFrame((zip(Images_List)), columns = ['Image'])

    return (df_img)

Start Chrome Driver to get ready to scrape

In [98]:
driver = webdriver.Chrome(chromedriver)

### Time to scrape!  

When running the cell below, you should get a new output message every 30-40 seconds to let you know the progress. When everything has finished running you will see an output of "Final Data Frames Saved"

In [99]:
type1 = 'poisonous'
num1 = 5
type2 = 'inedible'
num2 = 10
type3= 'edible'
num3 = 8
start_mushrooms(type1, num1, type2, num2, type3, num3)

cookies accepted
warning accepted
scraping poisonous page 1 out of 4
scraping poisonous page 2 out of 4
scraping poisonous page 3 out of 4
scraping poisonous page 4 out of 4
switching to inedible
scraping inedible page 1 out of 9
scraping inedible page 2 out of 9
scraping inedible page 3 out of 9
scraping inedible page 4 out of 9
scraping inedible page 5 out of 9
scraping inedible page 6 out of 9
scraping inedible page 7 out of 9
scraping inedible page 8 out of 9
scraping inedible page 9 out of 9
switching to edible
scraping edible page 1 out of 7
scraping edible page 2 out of 7
scraping edible page 3 out of 7
scraping edible page 4 out of 7
scraping edible page 5 out of 7
scraping edible page 6 out of 7
scraping edible page 7 out of 7
Final Data Frames Saved


In [100]:
mushrooms = pd.read_csv('mushrooms.csv')
mushrooms.head()

,Unnamed: 0,Species_Name,Family,Location,Dimensions,Edibility,Description
0,0,\n\n Amanita bisporigera\n ...,Amanitaceae,"North America, Mexico","Cap 3-10 cm diameter, stem 6-14 cm tall * 0.7-...",Lethally poisonous,"Amanita bisporigera, commonly known as Eastern..."
1,1,\n\n Amanita phalloides\n ...,Amanitaceae,"North America, Europe","Cap 6-12 cm diameter, stem 7-12 cm tall * 1 - ...",Lethally poisonous,"This is a lethally poisonous, medium to large,..."
2,2,\n\n Amanita virosa\n ...,Amanitaceae,"North America, Europe","Cap 5-9 cm diameter, stem 13-20 cm tall * 1.5-...",Lethally poisonous,"Amanita virosa, also known as Destroying angel..."
3,3,\n\n Cortinarius orellanus\n ...,Cortinariaceae,"North America, Europe","Cap 3-7 cm diameter, stem 3-8 cm tall * 0.5-1....",Lethally poisonous,"Cortinarius orellanus, also know as Fool's Web..."
4,4,\n\n Cortinarius rubellus\n ...,Cortinariaceae,Europe and parts of Asia,"Cap 3-8 cm diameter, stem 5-11 cm tall * 0.8-1...",Lethally poisonous,"Cortinarius rubellus, commonly known as the De..."


In [106]:
mushrooms = mushrooms.replace(r'\n',' ', regex=True) 

In [107]:
mushrooms

,Unnamed: 0,Species_Name,Family,Location,Dimensions,Edibility,Description
0,0,Amanita bisporigera ...,Amanitaceae,"North America, Mexico","Cap 3-10 cm diameter, stem 6-14 cm tall * 0.7-...",Lethally poisonous,"Amanita bisporigera, commonly known as Eastern..."
1,1,Amanita phalloides ...,Amanitaceae,"North America, Europe","Cap 6-12 cm diameter, stem 7-12 cm tall * 1 - ...",Lethally poisonous,"This is a lethally poisonous, medium to large,..."
2,2,Amanita virosa ...,Amanitaceae,"North America, Europe","Cap 5-9 cm diameter, stem 13-20 cm tall * 1.5-...",Lethally poisonous,"Amanita virosa, also known as Destroying angel..."
3,3,Cortinarius orellanus ...,Cortinariaceae,"North America, Europe","Cap 3-7 cm diameter, stem 3-8 cm tall * 0.5-1....",Lethally poisonous,"Cortinarius orellanus, also know as Fool's Web..."
4,4,Cortinarius rubellus ...,Cortinariaceae,Europe and parts of Asia,"Cap 3-8 cm diameter, stem 5-11 cm tall * 0.8-1...",Lethally poisonous,"Cortinarius rubellus, commonly known as the De..."
...,...,...,...,...,...,...,...
168,1,Tricholoma sejunctum ...,Tricholomataceae,"North America, Europe",Cap 4-10 cm diameter,Inedible,"Tricholoma sejunctum, also known as False Edib..."
169,2,Tricholomopsis decora ...,Tricholomataceae,"North America, Europe","Cap 4-12 cm diameter, stem 3.5-5.5 cm tall * 1...",Inedible,"Tricholomopsis decora, also known as Prunes an..."
170,3,Tricholomopsis rutilans ...,Tricholomataceae,"North America, Europe",Cap 2-12 cm diameter; stem 3.5-5.5 cm tall * 1...,Inedible,"Tricholomopsis rutilans, also known as Plums a..."
171,4,Turbinellus floccosus ...,Gomphaceae,"North America, Asia",5-15 cm diameter * 8-15 cm tall,Inedible,"Turbinellus floccosus, also known as Scaly Cha..."


In [110]:
mushrooms.loc[mushrooms['Edibility'] == 'poisonous', 'Species_Name']

Series([], Name: Species_Name, dtype: object)

In [101]:
mushrooms_image_links = pd.read_csv('mushrooms_img.csv')
mushrooms_image_links.head()

,Unnamed: 0,Image,edibility
0,0,/images/poisonous32.png,posionous
1,1,/../data/fungi/Amanitabisporigera1-thumb.jpg,posionous
2,2,/../data/fungi/Amanitabisporigera2-thumb.jpg,posionous
3,3,/../data/fungi/Amanitabisporigera3-thumb.jpg,posionous
4,4,/images/poisonous32.png,posionous


In [111]:
mushrooms_image_links = mushrooms_image_links.replace('/../data/fungi/',' ', regex=True) 

In [114]:
mushrooms_image_links = mushrooms_image_links.replace('-thumb.jpg',' ', regex=True) 

In [115]:
mushrooms_image_links

,Unnamed: 0,Image,edibility
0,0,/images/poisonous32.png,posionous
1,1,Amanitabisporigera1,posionous
2,2,Amanitabisporigera2,posionous
3,3,Amanitabisporigera3,posionous
4,4,/images/poisonous32.png,posionous
...,...,...,...
1012,28,Tylopilusfelleus2,inedible
1013,29,Tylopilusfelleus3,inedible
1014,30,Tylopilusfelleus4,inedible
1015,31,Tylopilusfelleus5,inedible


In [124]:
posionous = mushrooms_image_links.loc[mushrooms_image_links['edibility'] == 'posionous', 'Image']

In [137]:
posionous = posionous.str[:-1]

In [138]:
posionous

0          /images/poisonous32.pn
1             Amanitabisporigera1
2             Amanitabisporigera2
3             Amanitabisporigera3
4          /images/poisonous32.pn
                  ...            
192     Tricholomastiparophyllum1
193     Tricholomastiparophyllum2
194     Tricholomastiparophyllum3
195     Tricholomastiparophyllum4
196     Tricholomastiparophyllum5
Name: Image, Length: 197, dtype: object

In [140]:
def unique(list1):
 
    # initialize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    # print list
    for x in unique_list:
        print (x)

In [141]:
unique(posionous)

/images/poisonous32.pn
 Amanitabisporigera1
 Amanitabisporigera2
 Amanitabisporigera3
 Amanitaphalloides1-thumb.JP
 Amanitaphalloides2
 Amanitaphalloides3
 Amanitaphalloides4
 Amanitaphalloides5
 Amanitavirosa1
 Amanitavirosa2
 Amanitavirosa3
 Amanitavirosa4
 Amanitavirosa5
 Amanitavirosa6
 Amanitavirosa7
 Cortinariusorellanus1
 Cortinariusorellanus2
 Cortinariusrubellus1
 Cortinariusrubellus2
 Cortinariusrubellus3
 Galerinamarginata1
 Galerinamarginata2
 Galerinamarginata3
 Galerinamarginata4
 Galerinamarginata5
 Galerinamarginata6
 Gyromitraesculenta1
 Gyromitraesculenta2
 Gyromitraesculenta3
 Gyromitraesculenta4
 Gyromitraesculenta5
 Gyromitraesculenta6
 Rubroboletussatanas1
 Rubroboletussatanas2
 Amanitacokeri1
 Amanitacokeri2
 Amanitacokeri3
 Amanitacokeri4
 Amanitamuscaria1
 Amanitamuscaria2
 Amanitamuscaria3
 Amanitamuscaria4
 Amanitamuscaria5
 Amanitamuscaria6
 Amanitamuscaria7
 Amanitapantherina1
 Amanitapantherina2
 Amanitapantherina3
 Amanitaporphyria1
 Amanitaporphyria2
 Am

In [102]:
print("Text DF Info",mushrooms.info)
print("Image Link DF Info", mushrooms_image_links.info)

Text DF Info <bound method DataFrame.info of      Unnamed: 0                                       Species_Name  \
0             0  \n\n            Amanita bisporigera\n         ...   
1             1  \n\n            Amanita phalloides\n          ...   
2             2  \n\n            Amanita virosa\n              ...   
3             3  \n\n            Cortinarius orellanus\n       ...   
4             4  \n\n            Cortinarius rubellus\n        ...   
..          ...                                                ...   
168           1  \n\n            Tricholoma sejunctum\n        ...   
169           2  \n\n            Tricholomopsis decora\n       ...   
170           3  \n\n            Tricholomopsis rutilans\n     ...   
171           4  \n\n            Turbinellus floccosus\n       ...   
172           5  \n\n            Tylopilus felleus\n           ...   

               Family                  Location  \
0         Amanitaceae     North America, Mexico   
1         Am

In [103]:
mushrooms_image_links.groupby('edibility').count()

,Unnamed: 0,Image
edibility,,
edible,377,377
inedible,443,443
posionous,197,197


In [104]:
mushrooms_image_links.groupby('Image').count()

,Unnamed: 0,edibility
Image,,
/../data/fungi/Agaricusarvensis1-thumb.jpg,1,1
/../data/fungi/Agaricusarvensis2-thumb.jpg,1,1
/../data/fungi/Agaricusarvensis3-thumb.jpg,1,1
/../data/fungi/Agaricusarvensis4-thumb.jpg,1,1
/../data/fungi/Agaricusarvensis5-thumb.jpg,1,1
...,...,...
/../data/fungi/Xerocomussubtomentosus5-thumb.jpg,1,1
/../data/fungi/Xerocomussubtomentosus6-thumb.jpg,1,1
/images/edible32.png,62,62


In [92]:
mushrooms_image_links.groupby('edibility').count()

,Unnamed: 0,Image
edibility,,
edible,388,388
inedible,410,410
